## NBA Draft Lottery Probabilities 

I recently registered for an analytics conference coming up in September. The closing keynote speaker will be Earvin "Magic" Johnson, former Lakers star and recently hired as their President of Basketball Operations. Having been hired earlier this year, one of the first tests of the new regime was the NBA draft. To succeed in making good selections, the players not only need to be evaluated well, but the target player must be available to draft. For the Lakers, the latter was the easy part through having the second overall pick in the draft. However, it took some chance to get that pick as they trailed the Suns in probability of getting that position.  

Draft order is decided through a lottery in which the worst 14 teams from the previous season all have a chance at the top three picks. This occurs through the use of 1,000 lottery balls which are allocated by record with more a greater chance of being picked given to teams with worse records. After the first three positions, the remaining order is dictated purely by record from worst to last. This only applies to the first round of selections. I read an [article](http://bleacherreport.com/articles/2705995-nba-draft-lottery-rules-how-the-process-works) by Alec Nathan in Bleacher Report about the process. Given the odds of being selected first, he also provided the probability that each team would have a pick in the top 3. I will attempt to replicate that calculate probability. 

Nathan, A. (2017, May 08). NBA Draft Lottery Rules: How the Process Works. Retrieved July 18, 2017, from http://bleacherreport.com/articles/2705995-nba-draft-lottery-rules-how-the-process-works

In [1]:
"""Demonstrate NBA Draft Lottery probability."""

import pandas as pd
import numpy as np

# Import data
# -   Ovr_Pick1_Prob is the probability of having the first pick
# -   BR_Top3_Prob is Bleacher Report's calculation of having a top 3 pick
br = pd.read_csv('nba_lottery.csv', header=0, index_col='Team')
br

,Record,Ovr_Pick1_Prob,BR_Top3_Prob
Team,,,
Boston Celtics (via Brooklyn Nets),20-62,0.250,0.643
Phoenix Suns,24-58,0.199,0.558
Los Angeles Lakers,26-56,0.156,0.469
Philadelphia 76ers,28-54,0.119,0.378
Orlando Magic,29-53,0.088,0.291
Minnesota Timberwolves,31-51,0.053,0.183
New York Knicks,31-51,0.053,0.183
Sacramento Kings,32-50,0.028,0.099
Dallas Mavericks,33-49,0.017,0.061


## Calculations

Given that $p_x = probability\ team\ x\ drafts\ first\ overall$

$P(team\ i\ picks\ second)=\sum_{j}P(team\ i\ picks\ second\ |\ team\ j\ picks\ first)*P(team\ j\ picks\ first)=\sum_{j} \frac{p_i}{1-p_j}p_j$

This can be extended for the third pick:

$P(team\ i\ picks\ third)=\sum_{j}\sum_{k} \frac{p_i}{1-p_j-p_k}p_j\frac{p_k}{1-p_j}$

In [2]:
probs = br['Ovr_Pick1_Prob'].values

# Initialize storage of top 3 pick probability
probs_final = []

# Loop through teams
for i, _prob in enumerate(probs):

    # Create a copy of the list containing only other teams
    probs2 = probs.copy()
    probs2 = np.delete(probs2, i)

    # Probabilities of picking in each round
    p1 = probs[i]
    p2 = 0
    p3 = 0

    for j, _prob2 in enumerate(probs2):
        # Use Bayes' Rule for second overall pick
        p2 = p2 + probs2[j]*(p1/(1-probs2[j]))

        # Copy of the probs for other teams not getting the first two picks
        probs3 = probs2.copy()
        probs3 = np.delete(probs3, j)

        for k, _prob3 in enumerate(probs3):
            # Use Bayes' Rule for third overall pick
            numerator = (probs2[j]*probs3[k]/(1 - probs2[j]))*p1
            denominator = (1 - probs2[j] - probs3[k])
            p3 = p3 + numerator/denominator
    probs_final.append(round(p1+p2+p3, 3))

In [3]:
# Merge calculations to input data
df = pd.DataFrame(probs_final, index=br.index.tolist(),
                  columns=['AK_Top3_Prob'])
br.join(df)

,Record,Ovr_Pick1_Prob,BR_Top3_Prob,AK_Top3_Prob
Team,,,,
Boston Celtics (via Brooklyn Nets),20-62,0.250,0.643,0.643
Phoenix Suns,24-58,0.199,0.558,0.558
Los Angeles Lakers,26-56,0.156,0.469,0.469
Philadelphia 76ers,28-54,0.119,0.378,0.378
Orlando Magic,29-53,0.088,0.291,0.291
Minnesota Timberwolves,31-51,0.053,0.183,0.183
New York Knicks,31-51,0.053,0.183,0.183
Sacramento Kings,32-50,0.028,0.099,0.099
Dallas Mavericks,33-49,0.017,0.061,0.061


## Conclusion & What Really Happened

My calculations matched! That is much more involved that it looks on the surface. I was surprised at how low the probability of picking in the top three is for the first lottery slot. This makes me consider the strategy of tanking to get a high draft pick risky before even picking a player, a process which has its own risk. 

In reality, Boston received the first pick with Los Angeles and Philadelphia following in the next two slots. Phoenix was bumped to number four due. Boston and Philadelphia traded their picks before the draft.